In [99]:

from __future__ import unicode_literals
from nltk.corpus import stopwords
from mock import MagicMock

import pytest
import collections
import twitter
import operator
import os
import requests

def word_counter(text, language = 'spanish'):
    
    if(type(text) is not str or type(language) is not str):
        raise(TypeError)
        
    punctuation_marks = ["?", "¿", "¡", "!", " ", ",", ".", ";", ":","/"]
    
    filtred_text = ""
    
    for letter in text:
        if(letter not in punctuation_marks):
            filtred_text = filtred_text + letter
        else:
            filtred_text = filtred_text + " "
        
    filtred_text = filtred_text.lower().split()
    stop_words = set(stopwords.words(language))
    
    result_words = []
    trash_items = ["https","\\t","co",""]
    
    for word in filtred_text:
        if(word not in stop_words and word not in trash_items):
            result_words.append(word)
    
    if(len(result_words) == 0):
        return None
    
    return collections.Counter(result_words)
    

print(word_counter("Я собираюсь в путешествие",'russian'))


api = twitter.Api('gz2EucWLrJHTX2GjuMFxYN2la','e0vmSGeIlnHWbVGPO2YbfiPMUiZXh9DQDBML2fu0tqOoqylUXx','1115702759888523265-bbLQl3rRdu9beHs1UoyRXUZZfkqWv6','EttVVAmzpgvd6wnSO596xUIEzL7zGmqptXUQm6D2IACKS')

def test_tweets_words_counter_several_tweets():
    tweets = ['En tanto en cuanto nos den lo que es nuestro, discutiremos ese concepto con el fin de discutirlo…', 
              'El concepto, es el concepto',
              'El señor Villambrosa, que es un "gentelmán", me dijo que viniera a resolver esto con "pacisfismo", así que A, lo mismo que le digo una cosa le digo la otra, y B…, cuidado, que igual te viene la C…',
              'Bueno, vamos a llevarnos bien porque si no van a haber hondonadas de hostias aquí, ¿eh?',
              '¿No le he dicho ya que soy abogao? Payaso... ¡Idiota!',
              'El concepto, es el concepto',
              'Los hijos estudiando la carrera en los mejores colegios de Kanfort... con aprovechamiento, ¿eh? ¡Ay, Carmiña: tú en Cambados y yo en el País Vasco...!',
              'El concepto, es el concepto',
              'Sí, sí, el señor Villambrosa lo dejó muy claro. Si iba a venir, si no iba a venir... Vamos, que lo dejó clarísimo.',
              'El concepto, es el concepto',
              'Villambrosa, cuidao con el, ¿eh? Villambrosa es un cabrón, tiene mucha pasta... Nada en la ambulancia.']
    twitter_counter = twitter_word_counter(None,None)
    twitter_counter.get_last_50_tweets = MagicMock(return_value=tweets)
    assert twitter_counter.top_20_repetitive_word_counter() == [('concepto', 9), ('villambrosa', 4), ('eh', 3), ('si', 3), ('venir', 2), ('iba', 2), ('dejó', 2), ('vamos', 2), 
                                                                ('digo', 2), ('señor', 2), ('ambulancia', 1), ('pasta', 1), ('mucha', 1), ('cabrón', 1), ('cuidao', 1), ('clarísimo', 1), 
                                                                ('claro', 1), ('vasco', 1), ('país', 1), ('cambados', 1)]
    
def test_tweets_words_counter_with_an_account_withouth_tweets():
    tweets = []
    twitter_counter = twitter_word_counter(None,None)
    twitter_counter.get_last_50_tweets = MagicMock(return_value=tweets)
    assert twitter_counter.top_20_repetitive_word_counter() == None
    
def test_tweets_words_counter_with_one_tweet():
    tweets = ["El concepto, es el concepto"]
    twitter_counter = twitter_word_counter(None,None)
    twitter_counter.get_last_50_tweets = MagicMock(return_value=tweets)
    assert twitter_counter.top_20_repetitive_word_counter() == [('concepto', 2)]
    
def test_tweets_words_counter_if_twitter_api_fails():
    twitter_counter = twitter_word_counter(None,None)
    twitter_counter.get_last_50_tweets = MagicMock(side_effect=requests.exceptions.HTTPError)
    with pytest.raises(requests.exceptions.HTTPError):
        twitter_counter.top_20_repetitive_word_counter()
        
class twitter_word_counter(object):
    
    def __init__(self,twitter_api,username):
        self.twitter_api = twitter_api
        self.username = username
    
    def get_last_50_tweets(self):
        
        try:
            tweets = self.twitter_api.GetUserTimeline(screen_name = self.username,count = 400, include_rts=False)
        except: 
            return []
        tweets_text = []
        max_tweets = 50
        for i in range(0,len(tweets)):
            if(i is max_tweets):
                break
            tweets_text.append(tweets[i].text)
            
        return tweets_text
    
    def parse_list_tweets_to_string(self,tweets_list):
        
        if(type(tweets_list) is not list):
            raise(TypeError)
            
        tweets_string = ""
        for tweet in tweets_list:
            tweets_string = tweets_string + " " + tweet
        
        return tweets_string
    
    def top_20_repetitive_word_counter(self):
        tweets_text = self.get_last_50_tweets() 

        if(len(tweets_text) is 0):
            return None
        tweets_text = self.parse_list_tweets_to_string(tweets_text)
        values = word_counter(tweets_text)
        
        sorted_list = sorted(values.items(), key=operator.itemgetter(1))
        sorted_list.reverse()
        
        top_20_values = []
        stop_counter_condition = 0
        for i in range(0,len(sorted_list)):
            if(stop_counter_condition==20):
                break
            top_20_values.append(sorted_list[i]) 
            stop_counter_condition += 1 
            
        return top_20_values
        
        
        
        
modelo = twitter_word_counter(api, 'manolocabezabolo')
hola = modelo.top_20_repetitive_word_counter()
print(hola)
print(test_tweets_words_counter_if_twitter_api_fails())

Counter({'собираюсь': 1, 'путешествие': 1})
None
None
